<a href="https://colab.research.google.com/github/tmobley96/AI-SceneGen/blob/main/PEFTTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [2]:
!pip install -q bitsandbytes datasets accelerate loralib einops
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
!nvidia-smi -L


GPU 0: Tesla T4 (UUID: GPU-f47919a9-1eb6-0cb0-2c6e-0bb3e81184c3)


### Setup the model

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "cognitivecomputations/dolphin-2_6-phi-2",
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("cognitivecomputations/dolphin-2_6-phi-2")

config.json:   0%|          | 0.00/782 [00:00<?, ?B/s]

The repository for cognitivecomputations/dolphin-2_6-phi-2 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/cognitivecomputations/dolphin-2_6-phi-2.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/cognitivecomputations/dolphin-2_6-phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


The repository for cognitivecomputations/dolphin-2_6-phi-2 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/cognitivecomputations/dolphin-2_6-phi-2.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/584 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Freezing the original weights


In [7]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


# Setting up the LoRa Adapters

In [8]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [9]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32, #attention heads
    lora_alpha=16, #alpha scaling
    target_modules=["Wqkv", "out_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 15728640 || all params: 2795412480 || trainable%: 0.5626590033682615


# Data

In [10]:
import transformers
from datasets import load_dataset
data = load_dataset("tmobley96/black_mirror_scripts_S1-5")

Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
data['train'][0]

{'Script ID': '101',
 'Title': 'The National Anthem',
 'Dialogue': '',
 'Scene': '1',
 'Timestamp': '00:00:03,800 --> 00:00:05,840'}

# Training

In [12]:
print("Number of training samples:", len(data['train']))


Number of training samples: 17467


In [13]:

trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        save_steps=500,
        max_steps=10000,
        learning_rate=2e-4,
        fp16=True,
        bf16=False,
        logging_steps=10,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

IndexError: ignored

In [14]:
try:
    trainer.train()
except IndexError as e:
    print(f"IndexError: {e}")
    print(f"Current batch index: {trainer.state.global_step}")
    # Include any other relevant information you might need
    raise e  # Re-raise the exception if you want to halt the script


IndexError: Invalid key: 14177 is out of bounds for size 0
Current batch index: 0


IndexError: ignored